In [54]:
import numpy as np
np.set_printoptions(precision=1)
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [55]:
k =10
ma = np.random.dirichlet(np.ones(k), size=1)
cba = np.random.dirichlet(np.ones(k), size=k)

In [56]:
mb = np.sum(cba * ma[:, np.newaxis], axis=0)

In [57]:
def entropy(x):
    return -np.sum(x * np.log(x)) - np.log(x.shape[0])

In [58]:
print(entropy(ma), entropy(mb), entropy(np.ones(k)/k))

1.7467355755 1.717802664 -4.4408920985e-16


# Distances 

In [63]:
import categorical_distance as categ

In [204]:
k =100
causal = categ.sample_joint(k=k, n=5, concentration=1, symmetric=True)
anti = causal.reverse()
intcausal = causal.intervention(on='effect',concentration=1,fromjoint=False)
intanti = intcausal.reverse()

In [205]:
dicausal = causal.scoredist(intcausal)
dianti = anti.scoredist(intanti)
delta = dianti - dicausal 
print(delta) # the anticausal model is advantaged
print((delta>0).sum())

[-17231.9 -11897.  -14001.7 -13912.9 -13771.9]
0


 Now Let's look at the contribution of each term

In [206]:
# marginal difference
dimargi = (np.sum((anti.sa - intanti.sa)**2, axis=1))
# causal mean deviation
dicaumean = (np.sum((intanti.sa - np.mean(causal.sba, axis=1))**2,axis=1))
# anticausal mean deviation
dianmean = (np.sum((causal.sa - np.mean(anti.sba, axis=1))**2, axis=1))
print(dimargi)
print(dicaumean)
print(dianmean, 'alpha')

[ 169.7  119.7  140.5  144.2  139.6]
[ 174.6  120.9  141.9  141.1  139.7]
[ 0.6  0.7  0.5  0.5  0.6] alpha


In [207]:
estimate = dimargi - k*(dicaumean - dianmean) # oooh sign error
print('difference ', delta.mean())
print('estimate   ', estimate.mean())

difference  -14163.1047877
estimate    -14163.1047877


# variance of logsumexp

In [196]:
ss = np.random.randn(99,100)

In [197]:
lge = categ.logsumexp(ss)

In [198]:
np.var(lge)

0.017512454027028036

In [199]:
np.var(np.amax(ss,axis=0))

0.24195816381631272

In [200]:
np.var(np.mean(ss, axis=0))

0.010927555382498953

In [201]:
np.var(ss[0])

0.8400310407532311

In [202]:
np.var(ss[0] - np.mean(ss[1:],axis=0))

0.8466488010108999

In [211]:
print(np.mean(ss, axis=0))
print(np.round(causal.sba.mean(1)[0],decimals=1))

[-0.  -0.1  0.  -0.1  0.  -0.1 -0.1 -0.  -0.2 -0.1  0.1 -0.2 -0.3 -0.   0.
 -0.1  0.  -0.1 -0.1 -0.1  0.   0.   0.1 -0.1  0.  -0.   0.  -0.1  0.1 -0.
 -0.1 -0.   0.2 -0.   0.1  0.2 -0.2 -0.   0.   0.1  0.1  0.   0.1 -0.   0.3
 -0.1 -0.   0.1  0.2  0.   0.1  0.1  0.2  0.  -0.1  0.1 -0.1  0.1 -0.3  0.
  0.2 -0.1 -0.1  0.2  0.1 -0.   0.1  0.1  0.1  0.1  0.  -0.  -0.2 -0.1  0.1
  0.  -0.1 -0.  -0.  -0.1 -0.1 -0.1 -0.1 -0.2 -0.1  0.1 -0.  -0.1  0.  -0.1
  0.  -0.1  0.   0.1  0.1 -0.1  0.   0.1  0.   0.1]
[-0.  -0.1 -0.1 -0.1 -0.1 -0.1 -0.1  0.2 -0.   0.1 -0.   0.  -0.1  0.2  0.
 -0.1 -0.1 -0.  -0.1 -0.1 -0.1  0.   0.  -0.  -0.  -0.1 -0.1  0.2 -0.1 -0.1
 -0.  -0.   0.  -0.1  0.   0.   0.1  0.1  0.  -0.1  0.1  0.2 -0.   0.   0.
 -0.2  0.1 -0.1  0.1  0.   0.1 -0.  -0.1  0.1  0.  -0.   0.1  0.2  0.   0.1
  0.   0.   0.1  0.   0.   0.1 -0.  -0.3 -0.  -0.1  0.1  0.3 -0.   0.2  0.
  0.2 -0.1  0.  -0.1  0.  -0.  -0.2 -0.1 -0.1 -0.2  0.1  0.2  0.1 -0.1  0.1
 -0.1 -0.3  0.  -0.3  0.2  0.1 -0.  -0.1  

In [214]:
np.var(causal.sba.mean(1))

0.015800798719678929